In [1]:
# Necessary imports
import pandas as pd
import numpy as np
import math

In [2]:
# Load data from excel file
stock_transfers_aggregated = pd.read_excel('../data/stock_transfers_aggregated.xlsx')
sales_aggregated = pd.read_excel('../data/sales_aggregated.xlsx')

In [3]:
# Replace whitespace of column titles with underscores
stock_transfers_aggregated.columns = [title.replace(' ', '_') for title in stock_transfers_aggregated.columns]
sales_aggregated.columns = [title.replace(' ', '_') for title in sales_aggregated.columns]

In [4]:
stock_transfers_aggregated

,Simulation_Round,Simulation_Step,Storage_Location,Quantity
0,4,20,02N,14086
1,4,20,02S,10000
2,4,20,02W,16342
3,4,19,02S,9260
4,4,18,02N,16089
...,...,...,...,...
191,1,8,02S,5000
192,1,8,02W,5000
193,1,5,02N,2500
194,1,5,02S,2500


In [5]:
sales_aggregated

,Material_Description,Material_Number,Quantity,Cost,Net_Value
0,500g Nut Muesli,CC-F01,179816,161834.40,929827.70
1,500g Blueberry Muesli,CC-F02,233000,286590.00,968148.96
2,500g Raisin Muesli,CC-F04,158083,124885.57,768773.76
3,1kg Nut Muesli,CC-F11,171283,277478.46,816037.41
4,1kg Blueberry Muesli,CC-F12,184644,419141.88,985521.49
5,1kg Strawberry Muesli,CC-F13,155000,353400.00,821105.60


In [6]:
# Calculate margin/pc of each product
sales_aggregated = sales_aggregated.assign(Margin=lambda x: (x.Net_Value - x.Cost) / x.Quantity)

In [7]:
# Calculate average margin of all products
margin_sum = 0

for index, row in sales_aggregated.iterrows():
    margin_sum += row.Margin
    
average_margin = round(margin_sum / sales_aggregated.shape[0], 2)

In [8]:
average_margin

3.42

In [9]:
# Shipping cost per shipping == 1 000€
# Calculate the shipping quantity to cover the costs of shipping
coverage_quantity = int(math.ceil(1000 / average_margin))

In [10]:
coverage_quantity

293

In [11]:
# Check which stock transfers lost us money
non_coverage_transfers = stock_transfers_aggregated[stock_transfers_aggregated.Quantity <= coverage_quantity].sort_values(by='Quantity')
non_coverage_transfers

,Simulation_Round,Simulation_Step,Storage_Location,Quantity
102,3,4,02N,1


In [12]:
# Check a buffer of coverage + three times the coverage
non_buffer_transfers = stock_transfers_aggregated[stock_transfers_aggregated.Quantity <= (coverage_quantity * 4)].sort_values(by='Quantity')
non_buffer_transfers

,Simulation_Round,Simulation_Step,Storage_Location,Quantity
102,3,4,02N,1
61,3,18,02S,308
54,3,20,02N,447
55,3,20,02S,447
56,3,20,02W,448
180,1,17,02S,567
130,2,15,02S,614
170,2,1,02N,666
129,2,15,02N,768
14,4,15,02W,797


In [56]:
groupby = stock_transfers_aggregated
groupby['Round_Step'] = groupby['Simulation_Round'].astype(str) + groupby['Simulation_Step'].astype(str)
groupby = groupby.drop(columns=['Simulation_Round', 'Simulation_Step'])
groupby = groupby.groupby(['Round_Step', 'Storage_Location']).sum()

In [57]:
groupby

Quantity
Round_Step Storage_Location          
111        02N                   4750
           02S                   5000
           02W                   5250
114        02N                  17750
           02S                  20000
...                               ...
48         02S                   4728
           02W                   5235
49         02N                    996
           02S                   5754
           02W                   3527

[196 rows x 1 columns]

In [58]:
groupby_location = groupby.groupby(['Round_Step']).count()
groupby_location['Round_Step'] == ''

,Quantity
Round_Step,
111,3
114,3
116,3
117,1
118,3
...,...
45,3
46,3
47,3


In [40]:
groupby_test = groupby_test.groupby(['Round_Step']).count()
groupby_test

,Quantity
Round_Step,
111,3
114,3
116,3
117,1
118,3
...,...
45,3
46,3
47,3


In [82]:
for index, row in group_by_round_and_step.iterrows():
    print(row)

Quantity    2500
Name: (1, 5, 02N), dtype: int64
Quantity    2500
Name: (1, 5, 02S), dtype: int64
Quantity    2500
Name: (1, 5, 02W), dtype: int64
Quantity    5000
Name: (1, 8, 02N), dtype: int64
Quantity    5000
Name: (1, 8, 02S), dtype: int64
Quantity    5000
Name: (1, 8, 02W), dtype: int64
Quantity    4750
Name: (1, 11, 02N), dtype: int64
Quantity    5000
Name: (1, 11, 02S), dtype: int64
Quantity    5250
Name: (1, 11, 02W), dtype: int64
Quantity    17750
Name: (1, 14, 02N), dtype: int64
Quantity    20000
Name: (1, 14, 02S), dtype: int64
Quantity    14750
Name: (1, 14, 02W), dtype: int64
Quantity    22237
Name: (1, 16, 02N), dtype: int64
Quantity    27360
Name: (1, 16, 02S), dtype: int64
Quantity    19295
Name: (1, 16, 02W), dtype: int64
Quantity    567
Name: (1, 17, 02S), dtype: int64
Quantity    13065
Name: (1, 18, 02N), dtype: int64
Quantity    31816
Name: (1, 18, 02S), dtype: int64
Quantity    23416
Name: (1, 18, 02W), dtype: int64
Quantity    3760
Name: (1, 19, 02W), dtype: int6